In [19]:
# If you've git cloned this repo, run this cell to add `geochron` to your python path
import sys
sys.path.insert(0, '../')

In [26]:
import datetime as dt
import pydeck as pdk
from geostructures import Coordinate, GeoPoint, GeoCircle
#from geochron.visualization.folium import timehex_styledict, timehex_backgroundata, timehex_timestampedgeojson
from geochron import convert_chronnet
from geostructures.collections import  Track
from geostructures.geohash import H3Hasher


hasher = H3Hasher(resolution=10)

In [21]:
track = Track(
    [
        GeoPoint(Coordinate(-0.104154, 51.511920), dt=dt.datetime(2020, 1, 1, 8, 5)),
        GeoCircle(Coordinate(-0.096533, 51.511903)  ,100, dt=dt.datetime(2020, 1, 1, 9, 23)),
        GeoCircle(Coordinate(-0.096533, 51.511903),1000, dt=dt.datetime(2020, 1, 6, 9, 27)),
        GeoPoint(Coordinate(-0.083765, 51.514423), dt=dt.datetime(2020, 1, 1, 9, 44)),
        GeoPoint(Coordinate(-0.087478, 51.508595), dt=dt.datetime(2020, 1, 1, 10, 5)),
        GeoPoint(Coordinate(-0.087478, 51.508595), dt=dt.datetime(2020, 1, 1, 11, 7)),
    ]
)

[WARNING] geostructures: Datetime does not contain timezone information; Zulu/UTC time assumed. (this warning will not repeat)


In [29]:
chronnet_output = convert_chronnet(fcol=track,
time_delta= dt.timedelta(hours=1), hash_func= hasher.hash_collection, self_loops = True, mode = "directed")

In [41]:
edgelist = list(chronnet_output.edges.data('weight'))

In [42]:
edgelist

[('8a194ad3056ffff', '8a194ad3078ffff', 1),
 ('8a194ad3078ffff', '8a194ad30087fff', 1),
 ('8a194ad3078ffff', '8a194ad30097fff', 1),
 ('8a194ad3078ffff', '8a194ad3009ffff', 1),
 ('8a194ad3078ffff', '8a194ad300b7fff', 1),
 ('8a194ad3078ffff', '8a194ad30407fff', 1),
 ('8a194ad3078ffff', '8a194ad3040ffff', 1),
 ('8a194ad3078ffff', '8a194ad30417fff', 1),
 ('8a194ad3078ffff', '8a194ad3041ffff', 1),
 ('8a194ad3078ffff', '8a194ad30427fff', 1),
 ('8a194ad3078ffff', '8a194ad3042ffff', 1),
 ('8a194ad3078ffff', '8a194ad30437fff', 1),
 ('8a194ad3078ffff', '8a194ad30447fff', 1),
 ('8a194ad3078ffff', '8a194ad3044ffff', 1),
 ('8a194ad3078ffff', '8a194ad30457fff', 1),
 ('8a194ad3078ffff', '8a194ad3045ffff', 1),
 ('8a194ad3078ffff', '8a194ad30467fff', 1),
 ('8a194ad3078ffff', '8a194ad3046ffff', 1),
 ('8a194ad3078ffff', '8a194ad30477fff', 1),
 ('8a194ad3078ffff', '8a194ad30487fff', 1),
 ('8a194ad3078ffff', '8a194ad3048ffff', 1),
 ('8a194ad3078ffff', '8a194ad30497fff', 1),
 ('8a194ad3078ffff', '8a194ad304

In [45]:
import h3
def h3edge_to_coordinates(edge):
    edge_coordinate = (h3.h3_to_geo(edge[0]), h3.h3_to_geo(edge[1]), edge[2])
    return edge_coordinate

In [49]:
edgecoordinates = h3edge_to_coordinates(edgelist[0])

In [60]:
def convert_to_circledict(edgecoordinates, origin, destination):
    return {
        "start_lat": edgecoordinates[0][0],
        "start_lng": edgecoordinates[0][1],
        "end_lat": edgecoordinates[1][0],
        "end_lng": edgecoordinates[1][1],
        "width": edgecoordinates[2],
        "origin": origin,
        "destination": destination
    }

In [70]:
def network_arc_circle(network):
    circle_data = []
    edgelist = list(network.edges.data('weight'))
    for edge in edgelist:
        edgecoordinates = h3edge_to_coordinates(edge)
        dict_edge = convert_to_circledict(edgecoordinates, edge[0], edge[1])
        circle_data.append(dict_edge)
    return circle_data

In [71]:
data = network_arc_circle(chronnet_output)

In [72]:
"""
GreatCircleLayer
================

Plot of direct flights to and from San Francisco International Airport.

Origin is in green; destinations are in blue.

Adapted from the deck.gl documentation.
"""


import pandas as pd

GREAT_CIRCLE_LAYER_DATA = "https://raw.githubusercontent.com/visgl/deck.gl-data/master/website/flights.json"  # noqa

df = pd.read_json(GREAT_CIRCLE_LAYER_DATA)

# Use pandas to prepare data for tooltip
df["from_name"] = df["from"].apply(lambda f: f["name"])
df["to_name"] = df["to"].apply(lambda t: t["name"])

# Define a layer to display on a map
layer = pdk.Layer(
    "GreatCircleLayer",
    df,
    pickable=True,
    get_stroke_width=12,
    get_source_position="from.coordinates",
    get_target_position="to.coordinates",
    get_source_color=[64, 255, 0],
    get_target_color=[0, 128, 200],
    auto_highlight=True,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{from_name} to {to_name}"},
)
r.picking_radius = 10

r.to_html("great_circle_layer.html")

In [73]:

layer = pdk.Layer(
    "ArcLayer",
    data,
    pickable=True,
    get_source_position=["start_lng", "start_lat"],
    get_target_position=["end_lng", "end_lat"],
    get_source_color=[255, 0, 0],
    get_target_color=[0, 255, 0],
    getWidth='width',
    auto_highlight=True
)
# Set the viewport location
view_state = pdk.ViewState(latitude=50, longitude=-40, zoom=1, bearing=0, pitch=0)

# Render
r = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "{origin} to {destination}"},
)
r.picking_radius = 10

In [69]:
r

{
  "initialViewState": {
    "bearing": 0,
    "latitude": 50,
    "longitude": -40,
    "pitch": 0,
    "zoom": 1
  },
  "layers": [
    {
      "@@type": "ArcLayer",
      "autoHighlight": true,
      "data": [
        {
          "destination": "8a194ad3078ffff",
          "end_lat": 51.508434717353346,
          "end_lng": -0.0881735304316476,
          "origin": "8a194ad3056ffff",
          "start_lat": 51.51469632742728,
          "start_lng": -0.08372156433638074,
          "width": 1
        },
        {
          "destination": "8a194ad30087fff",
          "end_lat": 51.51248057703994,
          "end_lng": -0.08250645807164922,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30097fff",
          "end_lat": 51.51277933908878,
          "end_lng": -0.08424747372500155,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad3009ffff",
          "end_lat": 51.51167145436615,
          "end_lng": -0.08363991466159784,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad300b7fff",
          "end_lat": 51.51358845636242,
          "end_lng": -0.08311399783166543,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30407fff",
          "end_lat": 51.51478336057266,
          "end_lng": -0.09007851038406521,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad3040ffff",
          "end_lat": 51.513675492687476,
          "end_lng": -0.08947078642419021,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30417fff",
          "end_lat": 51.51508203138792,
          "end_lng": -0.09181974924428594,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad3041ffff",
          "end_lat": 51.51397416636534,
          "end_lng": -0.09121197923189703,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30427fff",
          "end_lat": 51.51559253014667,
          "end_lng": -0.08894502047013798,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad3042ffff",
          "end_lat": 51.51448466766076,
          "end_lng": -0.08833731581116008,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30437fff",
          "end_lat": 51.51589122019627,
          "end_lng": -0.09068626109653027,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.0881735304316476,
          "width": 1
        },
        {
          "destination": "8a194ad30447fff",
          "end_lat": 51.51226891790834,
          "end_lng": -0.08712198674343653,
          "origin": "8a194ad3078ffff",
          "start_lat": 51.508434717353346,
          "start_lng": -0.

In [25]:
data = [
    {
        "start_lng": 121.56447,
        "start_lat": 25.0339639,
        "end_lng": -74.0060152,
        "end_lat": 40.7127281,
    },
    # More data points...
]

,from,to,from_name,to_name
0,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Liverpool John Lenn...",San Francisco Int'l,Liverpool John Lennon
1,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Newcastle Int'l', '...",San Francisco Int'l,Newcastle Int'l
2,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Madinah Int'l', 'ab...",San Francisco Int'l,Madinah Int'l
3,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Sakirpasa', 'abbrev...",San Francisco Int'l,Sakirpasa
4,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Amarillo Int'l', 'a...",San Francisco Int'l,Amarillo Int'l
5,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Birmingham Int'l', ...",San Francisco Int'l,Birmingham Int'l
6,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Logan Int'l', 'abbr...",San Francisco Int'l,Logan Int'l
7,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Bourgas', 'abbrev':...",San Francisco Int'l,Bourgas
8,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Bremen', 'abbrev': ...",San Francisco Int'l,Bremen
9,"{'type': 'major', 'name': 'San Francisco Int'l...","{'type': 'major', 'name': 'Bristol Int'l', 'ab...",San Francisco Int'l,Bristol Int'l
